In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# 데이터 불러오기
data = pd.read_csv("./data/otto_train.csv") # Product Category
data.head() # 데이터 확인

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [4]:
'''
id: 고유 아이디
feat_1 ~ feat_93: 설명변수
target: 타겟변수 (1~9)
'''

'\nid: 고유 아이디\nfeat_1 ~ feat_93: 설명변수\ntarget: 타겟변수 (1~9)\n'

In [5]:
nCar = data.shape[0] # 데이터 개수
nVar = data.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 61878 nVar: 95


## 의미가 없다고 판단되는 변수 제거

In [6]:
data = data.drop(['id'], axis = 1) # id 제거

## 타겟 변수의 문자열을 숫자로 변환

In [7]:
mapping_dict = {"Class_1": 1,
                "Class_2": 2,
                "Class_3": 3,
                "Class_4": 4,
                "Class_5": 5,
                "Class_6": 6,
                "Class_7": 7,
                "Class_8": 8,
                "Class_9": 9}
after_mapping_target = data['target'].apply(lambda x: mapping_dict[x])

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [8]:
feature_columns = list(data.columns.difference(['target'])) # target을 제외한 모든 행
X = data[feature_columns] # 설명변수
y = after_mapping_target # 타겟변수
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(49502, 93) (12376, 93) (49502,) (12376,)


## 1. XGBoost

- XGBoost는 Gradient Bboosting과 완전히 동일한데 Gradient Boosting의 목적식에 Regularization term이 추가 된 것
- Gradient Boosting은 weak leaner 만들고, 거기에 발생하는 잔차에 대해서 다시 나머지 weak leaner들이 예측하고, 그렇게해서 합쳐진 weak leaner들이 strong learner가 됨
- XGBoost는 Regularization term이 있어서 overfitting 방지할 수 있는 알고리즘

In [9]:
# !pip install xgboost
import xgboost as xgb
import time
start = time.time() # 시작 시간 지정
# DMatrix : Data Matrix
xgb_dtrain = xgb.DMatrix(data = train_x, label = train_y) # 학습 데이터를 XGBoost 모델에 맞게 변환
xgb_dtest = xgb.DMatrix(data = test_x) # 평가 데이터를 XGBoost 모델에 맞게 변환

# xgboost, lightgbm, catboost 모두 동일하게 적용됨
xgb_param = {'max_depth': 10, # 트리 깊이
         'learning_rate': 0.01, # Step Size
         'n_estimators': 100, # Number of trees, 트리 생성 개수
         'objective': 'multi:softmax', # 목적 함수, label이 여러 개라서 multi:softmax 사용
        'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야함.
        # 현재 y 범주 개수 : 9개
    
# <참고> xgboost.readthedocs.io/en/latest/parameter.html  
# parameter 중요한 것
# binary:logistic -> auc 설정해줘야 함
# reg:logistic -> rmse, error 설정해줘야 함
# 'objective'와 'eval_metric'만 바꿔주면 됨
# 하고싶은 목적에 따라 내가 파라미터 조정할 수 있음
# binary을 할거냐, regression을 할거냐, multi classification을 할거냐에 따라 달라져야할 파라미터는
# 'objective'와 'eval_metric'(평가지표)만 달라지고 나머지는 공통적인 파라미터임!!

xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
xgb_model_predict = xgb_model.predict(xgb_dtest) # 평가 데이터 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, xgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

[13:15:26] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Accuracy: 76.67 %
Time: 31.32 seconds


In [10]:
xgb_model_predict
# label값 직접적으로 예측한 거임
# 현재 label값 1~9

array([5., 3., 6., ..., 9., 2., 7.], dtype=float32)

## 2. LightGBM

In [11]:
# !pip install lightgbm
import lightgbm as lgb
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
             # 주로 max_depth, min_data_in_leaf 등은 성능 차이가 심함
             # 그래서 이런 파라미터는 바꿔가면서 성능 확인할 필요 있음
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'objective': 'multiclass', # 목적 함수, 파라미터 이름이 약간씩 다름
            'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
            # lamba_l1, lambda_l2 : regularization term인데 이걸 높이면
            # regularization 효과가 더 커짐. 검증 데이터에 대해 overfitting 방지할 수 있음
            # 꼭 늘린다고해서 검증 데이터에 대해 좋아진다는 것은 아님
            # 학습데이터 내에서 regularization 효과를 가지게 하는 것이기 때문에

# 파라미터같은 경우 default 값이 있어서 따로 지정안해줘도되는데
# 꼭 필요한 특수한 경우에만 조금 바꿔

lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
lgb_model_predict = np.argmax(lgb_model.predict(test_x), axis = 1) # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측
# 예측할 때는 np.argmax()취해줘야함. 가장 확률값이 높은 label을 가져와서 predict로 사용
print("Accuracy: %.2f" % (accuracy_score(test_y, lgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

C:\Users\hyun\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Accuracy: 73.57 %
Time: 16.60 seconds


In [12]:
lgb_model.predict(test_x)
# 각 class에 대한 확률값을 내뱉음
# 이 class가 정답률이 확률이 ~~~다

array([[9.14195396e-16, 2.27101123e-02, 3.85432853e-01, ...,
        3.25763637e-02, 7.61804989e-02, 4.64683926e-02],
       [1.16264426e-15, 3.77262272e-02, 2.26739830e-01, ...,
        1.92812290e-01, 1.01826669e-01, 8.11988908e-02],
       [8.03061518e-16, 1.74144816e-02, 1.18616633e-01, ...,
        2.57008295e-02, 6.69196402e-02, 4.08194769e-02],
       ...,
       [8.35633463e-16, 5.03899675e-02, 1.67414994e-01, ...,
        4.81925245e-02, 1.02188157e-01, 3.95359380e-01],
       [1.01560525e-15, 2.13472780e-02, 4.60509516e-01, ...,
        3.32864538e-02, 8.46310485e-02, 5.16230380e-02],
       [8.92775603e-16, 2.04756016e-02, 1.31867900e-01, ...,
        4.13697353e-01, 1.44381292e-01, 4.53796283e-02]])

## 3. Catboost

In [13]:
# !pip install catboost
import catboost as cb
start = time.time() # 시작 시간 지정
cb_dtrain = cb.Pool(data = train_x, label = train_y) # 학습 데이터를 Catboost 모델에 맞게 변환
cb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'eval_metric': 'Accuracy', # 평가 척도
            'loss_function': 'MultiClass'} # 손실 함수, 목적 함수
cb_model = cb.train(pool = cb_dtrain, params = cb_param) # 학습 진행
cb_model_predict = np.argmax(cb_model.predict(test_x), axis = 1) + 1 # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측, 인덱스의 순서를 맞추기 위해 +1
print("Accuracy: %.2f" % (accuracy_score(test_y, cb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

0:	learn: 0.5907034	total: 1.46s	remaining: 2m 24s
1:	learn: 0.6356107	total: 2.53s	remaining: 2m 3s
2:	learn: 0.6411256	total: 3.72s	remaining: 2m
3:	learn: 0.6480344	total: 4.86s	remaining: 1m 56s
4:	learn: 0.6508222	total: 6.01s	remaining: 1m 54s
5:	learn: 0.6499939	total: 7.2s	remaining: 1m 52s
6:	learn: 0.6507818	total: 8.27s	remaining: 1m 49s
7:	learn: 0.6548422	total: 9.48s	remaining: 1m 49s
8:	learn: 0.6559533	total: 10.7s	remaining: 1m 48s
9:	learn: 0.6560947	total: 12s	remaining: 1m 47s
10:	learn: 0.6568421	total: 13.2s	remaining: 1m 46s
11:	learn: 0.6588219	total: 14.4s	remaining: 1m 45s
12:	learn: 0.6592259	total: 15.4s	remaining: 1m 43s
13:	learn: 0.6611248	total: 16.7s	remaining: 1m 42s
14:	learn: 0.6625591	total: 17.9s	remaining: 1m 41s
15:	learn: 0.6631853	total: 19.1s	remaining: 1m 40s
16:	learn: 0.6639328	total: 20.3s	remaining: 1m 39s
17:	learn: 0.6668821	total: 21.5s	remaining: 1m 37s
18:	learn: 0.6669630	total: 22.6s	remaining: 1m 36s
19:	learn: 0.6675286	total: 23

In [14]:
cb_model.predict(test_x)

array([[-0.35426047,  1.22109587,  0.44230101, ..., -0.1698448 ,
        -0.02059177, -0.2130643 ],
       [-0.07235138,  0.42535181,  0.20060428, ...,  0.21863604,
         0.2719157 ,  0.25089315],
       [-0.3315885 , -0.31862353, -0.31279765, ..., -0.29798357,
        -0.24018767, -0.32984969],
       ...,
       [ 0.05304325,  0.02500267, -0.14752573, ..., -0.20741963,
         0.12789417,  1.51166757],
       [-0.55093666,  1.7691278 ,  0.99746884, ..., -0.3420542 ,
        -0.49799871, -0.38136323],
       [-0.3033724 ,  0.09352675, -0.11808658, ...,  0.65825036,
         1.05515787, -0.20799899]])

- 이 데이터는 동일한 파라미터 기준으로 XGBoost의 성능 값이 가장 좋게 나왔음
- 데이터마다 성능이 달라지게 나오지만 일반적으론 lightGBM이랑 CatBoost가 성능이 좋음

In [15]:
# bagging할 때 집값예측 했었는데 lightGBM 이용해보자

# 데이터 불러오기
data = pd.read_csv("./data/kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [16]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

In [21]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


In [22]:
# !pip install lightgbm
import lightgbm as lgb
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 500, # Number of trees, 트리 생성 개수
            'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
            # 지금은 y가 price니까 regression 해야함 (실제 값(실수)를 예측하는거니까)
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행

In [23]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sqrt(mean_squared_error(lgb_model.predict(test_x),test_y))

# XGBoost, LightGBM, CatBoost는 기본적으로 seed값 유지(어떤 랜덤값 사용을 유지)
# 그래서 결과값이 계속 똑같이 나오는 것
# 원래 ensemble 모델(bagging도)은 랜덤하게 값을 냈었음, 결과가 계속 달라졌음  

210904.17249451784

In [25]:
import random
bagging_predict_result = [] # 빈 리스트 생성
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    print(len(set(random_data_index)))
    lgb_dtrain = lgb.Dataset(data = train_x.iloc[random_data_index,], label = train_y.iloc[random_data_index,]) # 학습 데이터를 LightGBM 모델에 맞게 변환
    lgb_param = {'max_depth': 14, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 500, # Number of trees, 트리 생성 개수
            'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
    lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
    predict1 = lgb_model.predict(test_x) # 테스트 데이터 예측
    bagging_predict_result.append(predict1) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장

9521
9566
9495
9551
9583
9567
9523
9547
9509
9606


In [26]:
bagging_predict_result

[array([517274.94966519, 634999.4998453 , 939184.0657075 , ...,
        342342.87263538, 881041.50928818, 460107.86834687]),
 array([486754.77428311, 628665.8815366 , 995850.41024787, ...,
        331383.67437888, 845178.51003488, 451320.78082687]),
 array([502556.76136649, 611312.67468108, 945901.17524381, ...,
        339950.7018672 , 953528.07214293, 457450.1405356 ]),
 array([486070.61428708, 675930.61652543, 899683.96763626, ...,
        353447.47408187, 948722.42660229, 472842.34766175]),
 array([499319.95299245, 602336.89455686, 979422.32923521, ...,
        361487.68030944, 921144.50775263, 477387.23459114]),
 array([511805.56177979, 626866.82229589, 984868.87893859, ...,
        332323.24069741, 876946.44995558, 469753.30674691]),
 array([507111.02053907, 579923.47405972, 937565.77804074, ...,
        341357.54838661, 896566.52907577, 472518.31265808]),
 array([531119.18583518, 650141.86020889, 906673.01724337, ...,
        334945.03391164, 905341.34931572, 461155.79656715]),


In [27]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict = [] # 빈 리스트 생성
for lst2_index in range(test_x.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [28]:
# 예측한 결과값들의 평균을 계산하여 실제 테스트 데이트의 타겟변수와 비교하여 성능 평가

print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y)))) # RMSE

RMSE: 211429.89289797368


In [29]:
bagging_predict

[505232.68556743243,
 630808.5517494406,
 942628.5075069293,
 1617842.4267852614,
 642904.5635938344,
 367359.46941588284,
 715472.4922366426,
 427248.14244067797,
 462685.11790992785,
 492322.5398335989,
 635659.141659116,
 377522.21080124634,
 302277.3177989128,
 357715.71936291485,
 343643.05016930436,
 1331816.2575478964,
 364034.83150364715,
 1005006.5662532825,
 319902.48356857576,
 526464.2064549784,
 372445.5048049033,
 1945442.6815031343,
 663310.8320664996,
 531640.4683656403,
 497986.9560764538,
 484643.14635907486,
 300249.6798621326,
 251552.61919876345,
 479487.398956017,
 537349.9239809252,
 488310.74620178354,
 469397.7665769254,
 469925.3599917645,
 569261.4329468954,
 373924.0429093299,
 1043878.4544649003,
 916557.6933115909,
 529537.4049177327,
 356176.66925292806,
 1520371.6170041172,
 394199.2812941662,
 276835.0189664132,
 510335.6656066074,
 340165.1986102342,
 256731.156341801,
 244631.41935874912,
 330244.1109232489,
 334660.30215882533,
 353465.3255016812,
 5